In [ ]:
# class LeNet:
# 	@staticmethod
# 	def build(width, height, depth, classes):
# 		# initialize the model
# 		model = Sequential()
# 		inputShape = (height, width, depth)

# 		# if we are using "channels first", update the input shape
# 		if K.image_data_format() == "channels_first":
# 			inputShape = (depth, height, width)

# 		# first set of CONV => RELU => POOL layers
# 		model.add(Conv2D(20, (5, 5), padding="same",
# 			input_shape=inputShape))
# 		model.add(Activation("relu"))
# 		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# 		# second set of CONV => RELU => POOL layers
# 		model.add(Conv2D(50, (5, 5), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# 		# first (and only) set of FC => RELU layers
# 		model.add(Flatten())
# 		model.add(Dense(500))
# 		model.add(Activation("relu"))

# 		# softmax classifier
# 		model.add(Dense(classes))
# 		# model.add(Activation("softmax"))

# 		# return the constructed network architecture
# 		return model
# class MiniVGGNet:
# 	@staticmethod
# 	def build(width, height, depth, classes):
# 		# initialize the model along with the input shape to be
# 		# "channels last" and the channels dimension itself
# 		model = Sequential()
# 		inputShape = (height, width, depth)
# 		chanDim = -1

# 		# if we are using "channels first", update the input shape
# 		# and channels dimension
# 		if K.image_data_format() == "channels_first":
# 			inputShape = (depth, height, width)
# 			chanDim = 1

# 		# first CONV => RELU => CONV => RELU => POOL layer set
# 		model.add(Conv2D(32, (3, 3), padding="same",
# 			input_shape=inputShape))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(Conv2D(32, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))

# 		# second CONV => RELU => CONV => RELU => POOL layer set
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))

# 		# first (and only) set of FC => RELU layers
# 		model.add(Flatten())
# 		model.add(Dense(512))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization())
# 		model.add(Dropout(0.5))

# 		# softmax classifier
# 		model.add(Dense(classes))
# 		#model.add(Activation("softmax"))

# 		# return the constructed network architecture
# 		return model

In [ ]:
def compute_ratio(org_width,org_height,resized_w,resized_h):
    return resized_w / org_width , resized_h / org_height

In [ ]:
faImgPath="./car_plate_fa/validation"

In [1]:
def getPersianPlates(end=500):
    for i,path in enumerate(paths.list_images(faImgPath)):
        if  end=="full" or i <end :
            yield path


In [ ]:
class MyVgg():
    @staticmethod
    def build(width,height,depth,classes):
        model = Sequential()
        model.add(InceptionResNetV2(weights="imagenet",
                        include_top=False, input_shape=(width, height, depth)
                       )
                 )        
        model.add(tfl.Flatten())
        model.add(tfl.Dense(256, activation="relu"))
        # model.add(Dropout(0.05))
        model.add(tfl.Dense(128, activation="relu"))
        model.add(tfl.Dense(64, activation="relu"))
        model.add(tfl.Dense(classes,activation="relu"))
        model.layers[0].trainable = False #becareful to change index 
        return model

In [3]:
def plate_boundary_model(height,width):
    preprocess_input=tf.keras.applications.inception_resnet_v2.preprocess_input
    
    input_shape = (height,width,3)
    
    base_model = tf.keras.applications.InceptionResNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet') 
    base_model.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape) 
    
    x = preprocess_input(inputs)
    x = base_model(x,training=False) #traning=False or True
    x =tfl.Flatten()(x)
    x=tfl.Dense(256,"relu")(x)
    x=tfl.Dense(128,"relu")(x)
    x=tfl.Dense(64,"relu")(x) 
    outputs = tfl.Dense(4,'relu')(x)    
    
    model = tf.keras.Model(inputs, outputs)
    
    return model    

In [4]:
def car_data_generator(end):
    preprocess_input=tf.keras.applications.inception_resnet_v2.preprocess_input
    def generator():
        for carNumber, carPath in enumerate(getPersianPlates(end=end)):
            # print("\r",carNumber,end="")
            img=cv2.imread(carPath)
            original_width=img.shape[1]
            original_height=img.shape[0]
        
            w_ratio,h_ratio=compute_ratio(original_width,original_height,resized_width,resized_height)
        
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xmlPath=faImgPath +"_labels" +"/" + carPath.split(os.path.sep)[-1].split(".")[0] +".xml"
            xmlFile=minidom.parse(xmlPath)
            xmin=int (xmlFile.getElementsByTagName("xmin")[0].firstChild.data) 
            ymin=int (xmlFile.getElementsByTagName("ymin")[0].firstChild.data) 
            xmax=int (xmlFile.getElementsByTagName("xmax")[0].firstChild.data) 
            ymax=int (xmlFile.getElementsByTagName("ymax")[0].firstChild.data)
            
        
            xmin=int( xmin * w_ratio )
            ymin=int( ymin * h_ratio )
            xmax=int( xmax * w_ratio )
            ymax=int( ymax * h_ratio )
        
            img=cv2.resize(img,(resized_width,resized_height)) 
            img=img.astype(np.float32)
            label=(xmin,ymin,xmax,ymax)
            yield img,label
    return generator

In [3]:
def get_splitted_dataset(end,BATCH_SIZE,split_size=0.1):
    train_size = int(end * (1 - split_size))
    dataset = tf.data.Dataset.from_generator(
    generator=car_data_generator(end),
    output_types=(tf.float32, tf.int32),  
    output_shapes=(
        tf.TensorShape([resized_height, resized_width, 3]),
        tf.TensorShape([4])  
    )
)

    train_dataset = dataset.take(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_dataset = dataset.skip(train_size).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    return train_dataset,val_dataset